# SageMaker Pipeline (pipeline.py) 로컬에서 Test 하기



# 1. 환경 설정 및 컨피그 파일 로딩

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
code_pipeline_serving_config_json_path = 'pipelines/ncf/src/code_pipeline_serving_config.json'
sm_pipeline_serving_config_json_path = 'pipelines/ncf/src/sm_pipeline_serving_config.json'

In [ ]:
from pipelines.ncf.src.common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))


### 필요한 설정 값 로딩

In [ ]:
import boto3
import sagemaker
import os

region = code_pipeline_serving_dict["region"]
bucket = code_pipeline_serving_dict["bucket"]
role = code_pipeline_serving_dict["code_build_service_arn"]
model_package_group_name = code_pipeline_serving_dict["model_package_group_name"]
project_prefix = code_pipeline_serving_dict["project_prefix"]
endpoint_name = code_pipeline_serving_dict["endpoint_name"]

pipeline_name = sm_pipeline_serving_dict["sm_pipeline_name"]
endpoint_instance_type = sm_pipeline_serving_dict["endpoint_instance_type"]
ModelApprovalStatus = sm_pipeline_serving_dict["ModelApprovalStatus"]


In [ ]:
print("project_prefix: \n", project_prefix)
print("region: \n", region)
print("role: \n", role)
print("bucket: \n", bucket)
print("model_package_group_name: \n", model_package_group_name)
print("endpoint_name: \n", endpoint_name)

print("ModelApprovalStatus: \n", ModelApprovalStatus)
print("pipeline_name: \n", pipeline_name)
print("endpoint_instance_type: \n", endpoint_instance_type)

# 3. Pipeline 테스트

In [ ]:
from pipelines.ncf.pipeline import get_pipeline


pipeline = get_pipeline(
    project_prefix = project_prefix,
    region = region,
    endpoint_name = endpoint_name, # Sagemaker Endpoint Name
    role= role, # SAGEMAKER_PIPELINE_ROLE_ARN 이 넘어옴.
    default_bucket=bucket,
    model_package_group_name= model_package_group_name,
    pipeline_name= pipeline_name
)

In [ ]:
definition = json.loads(pipeline.definition())
# definition

In [ ]:
pipeline.upsert(role_arn=role)
#execution = pipeline.start()
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus = ModelApprovalStatus,                                
        endpoint_instance_type = endpoint_instance_type
    )
)

In [ ]:
execution.wait()
execution.describe()

In [ ]:
execution.list_steps()